# Introduction 

- Following Code reads FVCOM particle tracking outputs and get the group_number
- Count the number if group_id in each group
- Add a new column to the data called group_number and add the number of particle in each group_id 
- The order is the first number is group_is,second numbers are particle number 


## Imports Libraries

In [1]:
# Importing libraries
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr

# Input Files

In [2]:
# Set the directory of the FVCOM model outputs
FVCOM_dir = '/home/abolmaal/modelling/FVCOM/Huron/output'
# Set the directory of the FVCOM model outputs
files = glob.glob(os.path.join(FVCOM_dir, 'FVCOM_Huron_2323_*.nc'))

# Output Files

# Helper Functions 

In [3]:
# Define the function to sort the files based on the time
def sort_key(file):
    filename = os.path.basename(file)
    try:
        # Extract the number after the double underscores and before the `.nc` extension
        number = int(filename.split('_')[-1].split('.')[0])
        return number
    except (IndexError, ValueError):
        # Handle filenames that do not match the pattern by returning a high number to place them last
        return float('inf')

# Updated Function: group_id + particle_index + yymmddHHMM

### Example Output
group_id = 1, group_index = 5, release_time = 2301 → particle_id = 001052301

In [4]:
import xarray as xr
import pandas as pd
import numpy as np
import os

def write_particle_id(files, data_dir):
    """
    Adds a particle_id = group_id(3-digit) + group_index(2-digit) + release_time(4-digit) to each NetCDF file.
    Also adds a group_number field if needed (optional).
    """

    release_times = [
        "2301", "2302", "2303", "2304",
        "2305", "2306", "2307", "2308",
        "2309", "2310", "2311", "2312"
    ]
    # release_times = [
    #     "2401", "2402", "2403", "2404",
    #     "2405", "2406", "2407", "2408",
    #     "2409", "2410", "2411", "2412"
    # ]

    for idx, file in enumerate(files):
        print(f"🔄 Processing: {os.path.basename(file)}")

        ds = xr.open_dataset(file)
        num_particles = ds.sizes['particles']

        # Get group_id values
        group_ids = ds['group_id'].values
        release_time = release_times[idx]

        # Create DataFrame
        df = pd.DataFrame({'group_id': group_ids})
        df['group_index'] = df.groupby('group_id').cumcount()

        # Create particle_id with padding
        df['particle_id'] = df.apply(
            lambda row: f"{int(row['group_id']):03}{int(row['group_index']):02}{release_time}",
            axis=1
        )

        # Safety check
        if len(df) != num_particles:
            raise ValueError(f"❌ Length mismatch in {file}: {len(df)} particle_ids vs {num_particles} particles")

        # Assign to dataset
        ds['particle_id'] = (('particles'), df['particle_id'].values.astype('U15'))

        # Save updated file
        output_path = os.path.join(data_dir, f"updated_{os.path.basename(file)}")
        ds.to_netcdf(output_path)
        print(f"✅ Saved: {output_path}\n")




# Main Functions

In [5]:
# Load the FVCOM output files and sort them based on the time
files = glob.glob(FVCOM_dir + "/FVCOM_Huron_2323_*.nc")
files.sort(key=sort_key)
print(files)


['/home/abolmaal/modelling/FVCOM/Huron/output/FVCOM_Huron_2323_JanFeb_1.nc']


In [6]:
# Call the function to update the NetCDF files with the group_number column
write_particle_id(files, FVCOM_dir)


🔄 Processing: FVCOM_Huron_2323_JanFeb_1.nc
✅ Saved: /home/abolmaal/modelling/FVCOM/Huron/output/updated_FVCOM_Huron_2323_JanFeb_1.nc



In [8]:
# Make sure the group_number column is added to the NetCDF files and is in right order 
# read the updated NetCDF file
updated_files = glob.glob(FVCOM_dir + "/updated_FVCOM_Huron_2323_*.nc")
updated_files.sort(key=sort_key)
# Open the first file
ds = xr.open_dataset(updated_files[0])

# Print the first 10 particle_id values
print("🔎 First 10 particle_id values:")
print(ds['particle_id'].values[:152])
# # read files usimg xarray
# ds = xr.open_mfdataset(updated_files, combine='nested', concat_dim='time', parallel=True)

# # read the firs file
# #ds = xr.open_dataset(updated_files[:])
# # print the updated group_number variable values
# print(ds['group_number'].values)

🔎 First 10 particle_id values:
['000002301' '000012301' '000022301' '000032301' '000042301' '000052301'
 '000062301' '000072301' '000082301' '000092301' '000102301' '000112301'
 '000122301' '000132301' '000142301' '000152301' '000162301' '000172301'
 '000182301' '000192301' '000202301' '000212301' '000222301' '000232301'
 '000242301' '000252301' '000262301' '000272301' '000282301' '000292301'
 '000302301' '000312301' '000322301' '000332301' '000342301' '000352301'
 '000362301' '000372301' '000382301' '000392301' '000402301' '000412301'
 '000422301' '000432301' '000442301' '000452301' '000462301' '000472301'
 '000482301' '000492301' '000502301' '000512301' '000522301' '000532301'
 '000542301' '000552301' '000562301' '000572301' '000582301' '000592301'
 '000602301' '000612301' '000622301' '000632301' '000642301' '000652301'
 '000662301' '000672301' '000682301' '000692301' '000702301' '000712301'
 '000722301' '000732301' '000742301' '003002301' '003012301' '003022301'
 '003032301' '003042

In [15]:
# write a function to read each file adn 1- # Step 1: Convert time to datetime (safe for large data)
# Step 2: Extract day and month period
# Step 3: Count unique days per month (memory-efficient)
# Step_ 4: Create a DataFrame with the results
def count_unique_days_per_month(files):
    """
    Counts unique days per month across multiple NetCDF files.
    Returns a DataFrame with the results.
    """
    results = []

    for file in files:
        print(f"🔄 Processing: {os.path.basename(file)}")
        with xr.open_dataset(file) as ds:  # Use context manager

            # Convert time to datetime
            time = pd.to_datetime(ds['time'].values, unit='s', origin='unix')

            # Extract day and month
            days = time.day
            months = time.month

            # Create a DataFrame for counting unique days per month
            df = pd.DataFrame({'day': days, 'month': months})
            unique_days_per_month = df.groupby('month')['day'].nunique().reset_index()

            # Append results
            results.append(unique_days_per_month)

    # Concatenate all results into a single DataFrame
    final_df = pd.concat(results, ignore_index=True)
    final_counts = final_df.groupby('month')['day'].sum().reset_index()
    # save the final counts to a CSV file
    output_csv = os.path.join(FVCOM_dir, 'days_per_releasetime_24.csv')
    final_counts.to_csv(output_csv, index=False)
    return final_counts

In [16]:
#  Call the function to count unique days per month
unique_days_per_month = count_unique_days_per_month(updated_files)

🔄 Processing: updated_FVCOM_Huron_2424_JanFeb_1.nc
🔄 Processing: updated_FVCOM_Huron_2424_FebMar_2.nc
🔄 Processing: updated_FVCOM_Huron_2424_MarApr_3.nc
🔄 Processing: updated_FVCOM_Huron_2424_AprMay_4.nc
🔄 Processing: updated_FVCOM_Huron_2424_MayJun_5.nc
🔄 Processing: updated_FVCOM_Huron_2424_JunJul_6.nc
🔄 Processing: updated_FVCOM_Huron_2424_JulAug_7.nc
🔄 Processing: updated_FVCOM_Huron_2424_AugSep_8.nc
🔄 Processing: updated_FVCOM_Huron_2424_SepOct_9.nc
🔄 Processing: updated_FVCOM_Huron_2424_OctNov_10.nc
🔄 Processing: updated_FVCOM_Huron_2424_NovDec_11.nc


In [ ]:
# print the occurance of each group_id in ds using xarray
group_id_counts = ds['group_id'].count(dim='particles').values

In [ ]:
# Extract the year and month as 'YYYY-MM' from 'time'
ds['month'] = ds['time'].dt.strftime('%Y-%m')

# Convert to a DataFrame for easier manipulation
df = ds.to_dataframe()

# Group by 'month' and 'group_id', and count the occurrences
group_id_counts_df = df.groupby(['month', 'group_id']).size().reset_index(name='group_id_count')

# Pivot the table to have months as rows and group_ids as columns
pivot_df = group_id_counts_df.pivot(index='month', columns='group_id', values='group_id_count').fillna(0)

# Print the resulting pivoted DataFrame
print(pivot_df)
#save this pivoted DataFrame to a csv file
output_csv_path = os.path.join(FVCOM_dir, 'group_id_counts_by_month.csv')
pivot_df.to_csv(output_csv_path)

In [ ]:
# Convert to a DataFrame for easier manipulation
df = ds.to_dataframe().reset_index()  # Resetting the index to make 'time' a column

# Check the first few rows to confirm the structure of the DataFrame
print(df.head())

# Filter the data for the specific day and group_id = 0
filtered_df = df[(df['month'] == '2023-01') & (df['group_id'] == 0)]

# Count the occurrences of group_id = 0 on the day 2023-01-01
occurrences = filtered_df[filtered_df['time'] == '2023-01-01'].shape[0]

# Print the result
print(f"Occurrences of group_id=0 on 2023-01: {occurrences}")

In [ ]:
# Convert to a DataFrame for easier manipulation
df = ds.to_dataframe().reset_index()  # Resetting the index to make 'time' a column

# Check the first few rows to confirm the structure of the DataFrame
print(df.head())

# Filter the data for group_id = 0
filtered_df = df[df['group_id'] == 0].copy()  # Use copy() to avoid warning

# Convert the 'time' column to just the date (ignoring the time part)
filtered_df['date'] = filtered_df['time'].dt.date  # Extract date without time

# Extract month and year for filtering
filtered_df['month'] = filtered_df['time'].dt.to_period('M')  # Use .loc to avoid warning

# Filter data for the month of January 2023
filtered_january = filtered_df[filtered_df['month'] == '2023-02']

# Now count the total occurrences of group_id = 0 for the entire month of January 2023
total_occurrences = filtered_january.shape[0]  # Count all rows

# Print the result
print(f"Total occurrences of group_id=0 in January 2023: {total_occurrences}")

In [ ]:
# Convert the NetCDF 'group_id' and 'group_number' variables to a DataFrame for processing
netcdf_df = ds['group_id'].to_dataframe().reset_index()

# Ensure that 'group_number' is extracted and correctly added
netcdf_df['group_number'] = ds['group_number'].values

# Step 1: Convert 'group_number' to integer if necessary, and format as a 5-digit string with leading zeros
netcdf_df['group_number'] = netcdf_df['group_number'].apply(lambda x: f"{int(x):11}")

# Step 2: Print values for debugging
print("First few rows of netcdf_df:")
print(netcdf_df.head())

# Step 3: Select relevant columns for saving to CSV
netcdf_df = netcdf_df[['group_id', 'group_number']]

# Save the result as a CSV file
netcdf_df.to_csv(os.path.join(FVCOM_dir, 'group_id_group_number.csv'), index=False)

print("CSV file saved with formatted group numbers")

